In [3]:
import os, json
from pymongo import MongoClient
from dotenv import load_dotenv

# READ-ONLY: do not create or modify any document.

def get_only_empty_form():
    load_dotenv()
    uri = os.getenv("MONGODB_URI")
    database_name = os.getenv("MONGODB_DATABASE")
    collection_name = "sales_invoices"

    if not uri or not database_name:
        print(json.dumps({
            "ok": False,
            "reason": "config_error",
            "message": "Missing MONGODB_URI or MONGODB_DATABASE"
        }, indent=2))
        return

    try:
        client = MongoClient(uri, serverSelectionTimeoutMS=5000)
        db = client[database_name]
        col = db[collection_name]

        docs = list(col.find({"is_empty": True}).sort("created", -1))

        if len(docs) == 0:
            print(json.dumps({
                "ok": False,
                "reason": "none_found",
                "message": "No document with is_empty=True found.",
                "count": 0
            }, indent=2, default=str))
            return

        if len(docs) > 1:
            sample = [
                {
                    "id": d.get("id"),
                    "current_form_qr_code": d.get("current_form_qr_code"),
                    "created": str(d.get("created")),
                    "_id": str(d.get("_id")),
                }
                for d in docs[:5]
            ]
            print(json.dumps({
                "ok": False,
                "reason": "multiple_found",
                "message": f"Found {len(docs)} documents with is_empty=True. Exactly one is required.",
                "count": len(docs),
                "sample": sample
            }, indent=2, default=str))
            return

        # Exactly one
        print(json.dumps({
            "ok": True,
            "reason": "exactly_one",
            "message": "Exactly one empty form found.",
            "document": docs[0]
        }, indent=2, default=str))

    except Exception as e:
        print(json.dumps({
            "ok": False,
            "reason": "runtime_error",
            "message": str(e)
        }, indent=2))

get_only_empty_form()


{
  "ok": true,
  "reason": "exactly_one",
  "message": "Exactly one empty form found.",
  "document": {
    "_id": "68ed2e5185573c9c4a8ff7b8",
    "id": "63a37fe5-d6c8-4182-9a40-63326688a9db",
    "previous_form_qr_code": "20251013",
    "current_form_qr_code": "20251013",
    "tin": "123-456-789",
    "location": "JEF Gas Station \u2013 Sikatuna Branch",
    "created": "2025-10-13T16:52:33.309952+00:00",
    "date": "2025-10-14",
    "is_empty": true
  }
}
